In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
impossible_allowed = True
model_checkpoint = "distilbert-base-uncased"
batch_size = 32

## Loading the Dataset

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_data.csv')
df.rename(columns = {'Unnamed: 0':'id'}, inplace = True)
df.rename(columns = {'Theme':'title'}, inplace = True)
df.rename(columns = {'Paragraph':'context'}, inplace = True)
df.rename(columns = {'Question':'question'}, inplace = True)
df.rename(columns = {'Answer_possible':'is_possible'}, inplace = True)
df.rename(columns = {'Answer_text':'text'}, inplace = True)
df.rename(columns = {'Answer_start':'answer_start'}, inplace = True)

In [ ]:
# Combine all answer related columns in a dictionary.
answers = df[['text', 'answer_start']].to_dict('index')
for i1 in answers:
    for i2 in answers[i1]:
        answers[i1][i2] = answers[i1][i2].strip('][').split("', ")
    if '' in answers[i1]['text']:
        answers[i1]['text'].remove('')
    for i3 in answers[i1]['text']:
        i3 = i3[1:-1] # Remove the first and last inverted commas.
    for i4 in range(0, len(answers[i1]['answer_start'])):
        if answers[i1]['answer_start'][i4] != '':
            answers[i1]['answer_start'][i4] = int(answers[i1]['answer_start'][i4])
        else:
            del answers[i1]['answer_start'][i4]

df.drop(['is_possible', 'text', 'answer_start'], axis = 1, inplace=True)
df['answers'] = pd.Series(answers)

In [ ]:
# Checking for mismatch examples.

# for row in df.itertuples(index = True):
#     context = getattr(row,'context')
#     answers = getattr(row, 'answers')
#     ans_text = answers['text']
#     ans_start = answers['answer_start']
#     if len(ans_text) == 0 and len(ans_start) == 0:
#         continue
#     elif len(ans_text) == 0:
#         print(context, ans_text, ans_start)
#     elif len(ans_start) == 0:
#         print(context, ans_text, ans_start)

#     text = ans_text[0]
#     start = ans_start[0]
#     context = context.lower()
#     context = context.replace('\\',' ')
#     context = context.replace('(',' ')
#     context = context.replace(')',' ')
#     context = context.replace(',',' ')
#     context = context.replace('"',' ')
#     context = context.replace('\'',' ')
#     context = context.replace('$', ' ')
#     context = context.replace('%', ' ')
#     context = context.replace('[', ' ')
#     context = context.replace(']', ' ')
#     context = context.replace(';', ' ')
#     context = context.replace(':', ' ')
#     context = context.replace('-', ' ')
#     context = context.replace('—', ' ')
#     context = context.replace('–', ' ')
#     context = context.replace('.', ' ')
#     list1 = context.strip().split()
#     for i in range(len(list1)):
#         list1[i] = list1[i].strip()
#     text = text.lower()
#     text = text.replace('\\',' ')
#     text = text.replace('(',' ')
#     text = text.replace(')',' ')
#     text = text.replace(',',' ')
#     text = text.replace('"',' ')
#     text = text.replace('%',' ')
#     text = text.replace('\'',' ')
#     text = text.replace('$', ' ')
#     text = text.replace('-', ' ')
#     text = text.replace('—', ' ')
#     text = text.replace('[', ' ')
#     text = text.replace(']', ' ')
#     text = text.replace(':', ' ')
#     text = text.replace(';', ' ')
#     text = text.replace('.',' ')
#     text = text.replace('–', ' ')
#     list2 = text.strip().split()
#     for i in range(len(list2)):
#         list2[i] = list2[i].strip()
    
#     check = all(item in list1 for item in list2)

#     if not check:
#         print(getattr(row, 'id'), context, getattr(row, 'question'), text)

In [ ]:
from datasets import Dataset

datasets = Dataset.from_pandas(df)
datasets = datasets.train_test_split(test_size=0.2)

## Preprocessing the training data

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
# Account for the special case for compatibility with other models where model expects padding on the left. Here, switch the order of the question and the context.
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    # Remove left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize examples with truncation and padding, but keep the overflows using a stride.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # One example might give several features if it has a long context.
    # We need a map from a feature to its corresponding example.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # Label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example.
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_train_data = datasets["train"].map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)
tokenized_val_data = datasets["test"].map(prepare_train_features, batched=True, remove_columns=datasets["test"].column_names)

  0%|          | 0/61 [00:00<?, ?ba/s]

  0%|          | 0/16 [00:00<?, ?ba/s]

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="all",
)

In [ ]:
from transformers import default_data_collator

# Use data collator that will batch processed examples together.
data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 60724
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 5694
  Number of trainable parameters = 66364418


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
trainer.save_model(f"/content/drive/MyDrive/{model_name}-trained")

## Evaluation

In [ ]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

To classify answers, use the score obtained by adding the start and end logits. Limit checking with a hyper-parameter we call `n_best_size`. Pick the best indices in the start and end logits and gather all the answers this predicts. After checking if each one is valid, we sort them by their score and keep the best one. And then sort the `valid_answers` according to their `score` and only keep the best one. To check a given span is inside the context (and not the question) and to get back the text inside we need to add two things to our validation features:
- the ID of the example that generated the feature (since each example can generate several features, as seen before);
- the offset mapping that will give us a map from token indices to character positions in the context.

That's why we will re-process the validation set with the following function, slightly different from `prepare_train_features`:

In [ ]:
n_best_size = 20

In [ ]:
def prepare_validation_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
validation_features = datasets["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["test"].column_names
)

In [ ]:
raw_predictions = trainer.predict(validation_features)

In [ ]:
# The 'Trainer' hides the columns that are not used by the model (here `example_id` and `offset_mapping` which is required for our post-processing), so we set them back.
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

For general features, we will need a map between examples and their corresponding features as we do know from which example they come from. Also, since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one.

In [ ]:
import collections

examples = datasets["test"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
from tqdm.auto import tqdm
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.wer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []
        
        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]

            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer.
        if not impossible_allowed:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(datasets["test"], validation_features, raw_predictions.predictions)

In [ ]:
from datasets import load_metric

metric = load_metric("squad_v2" if impossible_allowed else "squad")

In [ ]:
if impossible_allowed:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["test"]]
metric.compute(predictions=formatted_predictions, references=references)

In [ ]:
# Test output timings with minimal post-processing.

import torch

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

device = torch.device('cpu:1') # For cuda, inferenc time is ~15ms.
PATH = "/content/drive/MyDrive/distilbert-base-uncased-trained/pytorch_model.bin"
model.load_state_dict(torch.load(PATH))
model = model.to(device)

context = "Fryderyk's father, Nicolas Chopin, was a Frenchman from Lorraine who had emigrated to Poland in 1787 at the age of sixteen. Nicolas tutored children of the Polish aristocracy, and in 1806 married Justyna Krzyżanowska, a poor relative of the Skarbeks, one of the families for whom he worked. Fryderyk was baptized on Easter Sunday, 23 April 1810, in the same church where his parents had married, in Brochów. His eighteen-year-old godfather, for whom he was named, was Fryderyk Skarbek, a pupil of Nicolas Chopin. Fryderyk was the couple's second child and only son; he had an elder sister, Ludwika (1807–55), and two younger sisters, Izabela (1811–81) and Emilia (1812–27). Nicolas was devoted to his adopted homeland, and insisted on the use of the Polish language in the household."

questions = [
    "Who did Frédéric's father marry in 1806?",
    "Where was Fryderyk's father from?",
    "Nicolas tutored which children?",
    "Which language did Frédéric's father, Nicolas, insist on using in the household?",
    "What was Chopin's father's first name?",
    "Which contry was cricket invented in?"
]

time = 0

for question in questions:
    inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="pt")
    inputs = inputs.to(device)
    input_ids = inputs["input_ids"].tolist()[0] # the list of all indices of words in question + context

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids) # Get the tokens for the question + context

    start.record()
    answer_start_scores, answer_end_scores = model(**inputs).values()

    answer_start = torch.argmax(answer_start_scores)  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    end.record()
    time = time + start.elapsed_time(end)
    print(answer)
    

# Waits for everything to finish running
torch.cuda.synchronize()
print("Inference time per question is:", time/len(questions), "ms")

justyna krzyzanowska, a
lorraine who
the polish aristocracy, and
polish language in
nicolas chopin, was
[CLS]
Inference time per question is: 267.2719268798828 ms
